In [51]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time
import string

In [21]:
URL = "https://www.basketball-reference.com/players/a/"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

In [22]:
player_tds = soup.find_all(attrs={"data-stat":"player"})

In [24]:
player_tds.pop(0)

<th aria-label="Player" class="poptip sort_default_asc center" data-stat="player" scope="col">Player</th>

In [ ]:
url_endings = []
for td in player_tds:
    anchor = td.find("a")
    url_endings.append(anchor.get("href"))

In [47]:
all_game_stat_urls = []
for player_ending in url_endings:
    PLAYER_URL = f"https://www.basketball-reference.com/{player_ending}"
    player_page = requests.get(PLAYER_URL)
    player_soup = BeautifulSoup(player_page.content, "html.parser")
    player_season_url_endings = []
    for li in player_soup.find_all('li'):
        first_tag = next((child for child in li.children if child.name), None)
        if first_tag and first_tag.name == 'span' and first_tag.get_text(strip=True) == 'Game Logs':
            inner_tags = li.find_all('li')
            pattern = re.compile(r'^\d{4}-\d{2}$')
            for tag in inner_tags:
                game_season_anchor = tag.find('a')
                if(pattern.match(game_season_anchor.text)):
                    player_season_url_endings.append(game_season_anchor.get("href"))
    prefix = 'https://www.basketball-reference.com'
    player_season_urls = [prefix + item for item in player_season_url_endings]
    all_game_stat_urls = all_game_stat_urls + player_season_urls
    time.sleep(4)

In [54]:
with open('gamlogurlstest.json', 'w') as f:
    json.dump(all_game_stat_urls, f)